# Croupier - Architecture Documentation

**Backend Intern Assignment - The Wedding Company**

**Author:** Vibhor Srivastava  
**Institution:** SRM Institute of Science and Technology, NCR Campus  
**Submission Date:** December 13, 2025

---

## 📑 Table of Contents

1. **Architecture Overview** - System design, layer responsibilities, request lifecycle
2. **Directory Structure** - Project organization and module breakdown
3. **Key Design Decisions** - Multi-tenancy strategy, authentication, layered architecture
4. **Code Examples** - Implementation snippets for each architectural layer
5. **Architectural Questions** - Comprehensive answers to assignment questions:
   - Q1: Scalability limitations and mitigation strategies
   - Q2: Technology stack trade-offs (FastAPI, MongoDB, JWT, Collection-per-Tenant)
   - Q3: Alternative database multi-tenancy patterns (Shared Collection, Database-per-Tenant, Hybrid Strategy)
6. **Production Readiness** - Performance optimization, security hardening, scalability preparation
7. **Implementation Summary** - Architecture evaluation, growth path, deployment recommendations

---

## Technical Deep-Dive

This document provides a comprehensive architectural analysis of Croupier, including implementation details, design decisions, scalability considerations, and detailed answers to the architectural questions from the assignment.

> **Note:** For quick start instructions and API reference, see the main [README.md](README.md)

---

## 1. Architecture Overview

### 1.1 System Design

Croupier implements a multi-tenant organization management service using a **layered architecture pattern**. The design prioritizes separation of concerns, maintainability, and horizontal scalability.

### 1.2 Layer Responsibilities

| Layer | Component | Responsibility |
|-------|-----------|----------------|
| **Presentation** | FastAPI Routers | HTTP handling, request validation, response serialization |
| **Business Logic** | Services | Transaction orchestration, business rules enforcement |
| **Data Access** | Repositories | Database abstraction, CRUD operations |
| **Persistence** | MongoDB | Master database + per-tenant dynamic collections |

### 1.3 Request/Response Lifecycle

```
Incoming Request → Router (validate) → Service (business logic) → Repository (data access) → MongoDB
MongoDB → Repository (model mapping) → Service (transform) → Router (serialize) → Response
```

**Key Design Choice:** Stateless architecture with JWT-based authentication enables horizontal scaling without session management overhead.

## 2. Directory Structure

The project follows a modular, production-ready structure with clear separation of concerns:

```
app/
  models/schemas.py       - Pydantic models
  repositories/           - Data access layer
    organization_repository.py
    admin_repository.py
  routers/               - API endpoints
    organization.py
    admin.py
  security/              - JWT & password hashing
    jwt_handler.py
    password_handler.py
    dependencies.py
  services/              - Business logic
    organization_service.py
    auth_service.py
  config.py              - Configuration
  db.py                  - Database manager
main.py                  - Application entry
```

### 2.1 Layer Responsibilities

- **Routers:** HTTP request handling, input validation, response formatting
- **Services:** Business logic, orchestration, transaction management
- **Repositories:** Database operations, data access abstraction
- **Models:** Data validation, serialization/deserialization
- **Security:** Authentication, authorization, encryption

## 3. Key Design Decisions

### 3.1 Multi-Tenancy Strategy

**Decision:** Separate MongoDB collection per organization (`org_<name>`)

**Rationale:**
- **Data Isolation:** Complete separation prevents data leakage between tenants
- **Scalability:** Collections can be sharded independently based on tenant size
- **Performance:** Queries don't need tenant filters (WHERE org_id = ?), reducing overhead
- **Compliance:** Simplified GDPR/data residency compliance (delete collection = delete all tenant data)

**Trade-off:** Higher collection count vs. simpler queries and better isolation

### 3.2 Authentication Architecture

**Decision:** JWT tokens with embedded `admin_id` + `organization_id`

**Rationale:**
- **Stateless:** No server-side session storage required → horizontal scaling
- **Performance:** Single token validation vs. database session lookup
- **Multi-tenant:** Organization context encoded in token → automatic tenant routing

**Security Measures:**
- bcrypt with 12 rounds for password hashing
- JWT expiration enforced
- Dependencies validate tokens on protected endpoints

### 3.3 Layered Architecture

**Decision:** Strict layer separation (Router → Service → Repository)

**Benefits:**
- **Testability:** Each layer can be unit tested independently
- **Maintainability:** Changes to database logic don't affect business logic
- **Code Reuse:** Services can be used by multiple routers
- **Extensibility:** Easy to add new data sources (e.g., cache layer)

## 4. Key Code Implementations

### 4.1 Dynamic Collection Creation (db.py)

In [ ]:
class DatabaseManager:
    def get_org_collection(self, organization_name: str):
        """Get or create organization-specific collection"""
        collection_name = f"org_{organization_name}"
        return self.master_db[collection_name]
    
    def drop_org_collection(self, organization_name: str):
        """Delete organization collection and all its data"""
        collection_name = f"org_{organization_name}"
        self.master_db.drop_collection(collection_name)

### 4.2 Repository Pattern (organization_repository.py)

In [ ]:
class OrganizationRepository:
    def create(self, organization_data: Dict[str, Any]):
        """Create new organization with metadata"""
        organization_data['created_at'] = datetime.utcnow()
        result = self.collection.insert_one(organization_data)
        return self._serialize_document(organization_data)
    
    def find_by_name(self, organization_name: str):
        """Find organization by name"""
        doc = self.collection.find_one(
            {"organization_name": organization_name})
        return self._serialize_document(doc) if doc else None

### 4.3 Service Layer (organization_service.py)

In [ ]:
class OrganizationService:
    def create_organization(self, org_data):
        """Complete organization creation workflow"""
        # 1. Validate uniqueness
        if self.org_repo.exists(org_data.organization_name):
            raise HTTPException(400, "Organization exists")
        
        # 2. Create organization metadata
        created_org = self.org_repo.create({...})
        
        # 3. Create admin user with hashed password
        self.admin_repo.create({...})
        
        # 4. Create dynamic collection with indexes
        org_collection = self.db_manager.get_org_collection(...)
        org_collection.create_index("created_at")

### 4.4 JWT Token Creation (security/jwt_handler.py)

In [ ]:
class JWTHandler:
    @staticmethod
    def create_access_token(data: Dict[str, Any]):
        """Create JWT token with expiration"""
        to_encode = data.copy()
        expire = datetime.utcnow() + timedelta(minutes=60)
        to_encode.update({
            "exp": expire,
            "iat": datetime.utcnow()
        })
        return jwt.encode(to_encode, SECRET_KEY, algorithm="HS256")

### 4.5 Password Hashing (security/password_handler.py)

In [ ]:
class PasswordHandler:
    @staticmethod
    def hash_password(password: str) -> str:
        """Hash password using bcrypt with 12 rounds"""
        salt = bcrypt.gensalt(rounds=12)
        hashed = bcrypt.hashpw(password.encode(), salt)
        return hashed.decode('utf-8')
    
    @staticmethod
    def verify_password(plain: str, hashed: str) -> bool:
        """Verify password against hash"""
        return bcrypt.checkpw(plain.encode(), hashed.encode())

## 5. Architectural Analysis

### Question 1: Is this architecture scalable?

**Answer:** Yes, to a significant extent, but with important caveats and limitations.

**Scalability Strengths**
- **Horizontal Scaling:** Stateless JWT allows adding multiple API servers without session synchronization
- **Data Isolation:** Each organization has its own collection enabling targeted optimization and sharding
- **Independent Growth:** Organizations can grow independently without affecting others
- **FastAPI Performance:** Async capabilities handle high concurrency efficiently

**Scalability Challenges**
- **Collection Explosion:** MongoDB has practical limits at 100,000+ collections; metadata management becomes challenging
- **Namespace Limits:** MongoDB namespaces (database.collection) have size constraints
- **Schema Migrations:** Changes must be applied to N collections, increasing complexity
- **Resource Overhead:** Each collection has index metadata and file descriptors

**Mitigation Strategies**
- Collection Pooling: Group small/inactive organizations into shared collections
- Archival: Move inactive organizations to cold storage
- Monitoring: Track collection count and implement alerts at thresholds
- Hybrid Approach: Use per-collection for large tenants, shared for small ones

### Q2: Technology Stack Trade-offs

#### FastAPI
**Advantages:**
- High performance (comparable to Node.js/Go)
- Automatic OpenAPI documentation generation
- Native type safety with Pydantic
- Modern async/await support for I/O-bound operations

**Limitations:**
- Smaller ecosystem compared to Django/Flask
- Async paradigm adds complexity for simple CRUD
- Less mature ORM integrations

#### MongoDB
**Advantages:**
- Schema flexibility for evolving data models
- Horizontal scaling through sharding
- Fast write operations (document-based)
- Natural JSON representation for REST APIs

**Limitations:**
- No ACID transactions across collections (in standalone mode)
- Collection count limits at scale (≈100k collections)
- Application-layer referential integrity
- Higher memory footprint vs. SQL

#### Collection-per-Tenant Pattern
**Advantages:**
- **Security:** Complete data isolation between tenants
- **Performance:** No tenant_id filters needed in queries
- **Compliance:** Simple tenant deletion (drop collection)
- **Optimization:** Per-tenant indexing strategies

**Limitations:**
- Collection explosion beyond 10,000+ tenants
- Schema migrations require N operations
- Cross-tenant analytics complexity
- Metadata overhead per collection

#### JWT Authentication
**Advantages:**
- Stateless architecture (no session storage)
- Horizontal scaling without sticky sessions
- Self-contained authorization (admin_id + org_id in token)

**Limitations:**
- Cannot revoke tokens before expiration (without Redis/blocklist)
- Token size grows with claims
- Requires secure secret key management

### Q3: Alternative Database Multi-Tenancy Patterns

MongoDB supports three primary multi-tenancy strategies, each optimized for different scale and isolation requirements:

#### **Pattern 1: Shared Collection with Tenant ID**

All tenant data in a single collection with `tenant_id` discriminator.

```python
# Example document
{
  "_id": ObjectId("..."),
  "tenant_id": "org_acme",
  "field1": "value1"
}
```

**Characteristics:**
- **Pros:** Simple management, no collection limits, easy cross-tenant analytics, single schema migration
- **Cons:** Weak isolation, noisy neighbor effects, all queries require tenant filter, index bloat

**Use Case:** <1000 similar-sized tenants, frequent cross-tenant reporting, lower isolation needs

---

#### **Pattern 2: Database per Tenant**

Complete physical isolation with dedicated database per tenant.

```python
# Each tenant gets own database
- customer_acme_db
- customer_globex_db
- customer_initech_db
```

**Characteristics:**
- **Pros:** Maximum isolation, independent backups, database-level security, per-tenant tuning
- **Cons:** High operational complexity, connection pool challenges, MongoDB database limits (≈10k)

**Use Case:** Enterprise clients (HIPAA/SOC2), <100 high-value tenants, geographic data residency

---

#### **Pattern 3: Collection per Tenant (Current Implementation)**

Middle-ground approach balancing isolation and operational complexity.

```python
# Master database with dynamic collections
- master_db
  ├── organizations  (metadata)
  ├── admin_users    (authentication)
  ├── org_acme       (tenant data)
  ├── org_globex     (tenant data)
  └── org_initech    (tenant data)
```

**Characteristics:**
- **Pros:** Good isolation, no tenant_id filters, fast deletion, per-collection sharding
- **Cons:** Collection limits (≈100k), N schema migrations, cross-tenant query complexity

**Use Case:** 100-10,000 tenants, moderate isolation needs, operational efficiency balance

---

#### **Recommended: Hybrid Tiered Strategy**

For production systems serving diverse tenant profiles:

| Tier | Pattern | Criteria |
|------|---------|----------|
| **Enterprise** | Database per Tenant | >$10k/month, compliance requirements |
| **Professional** | Collection per Tenant | Mid-size businesses, standard isolation |
| **Starter** | Shared Collection | <100 users, cost-sensitive |

This approach optimizes for both isolation requirements and operational scalability.

### Production Evolution Path

For a production-ready system serving diverse customer segments, implement a **tiered hybrid strategy**:

```python
# Intelligent tenant routing
def get_tenant_storage_strategy(organization):
    if organization.tier == "enterprise":
        return DatabasePerTenant()  # Dedicated database
    elif organization.tier == "professional":
        return CollectionPerTenant()  # Current implementation
    else:
        return SharedCollection()  # Shared with tenant_id filter
```

**Benefits:**
- **Cost Optimization:** Free/starter users share infrastructure
- **Revenue Optimization:** Enterprise clients get dedicated resources
- **Operational Efficiency:** Manage 1000s of small tenants efficiently
- **Scalability:** Each tier scales independently

This approach is used by production SaaS platforms like Slack, Atlassian, and Salesforce.

## 6. Production Readiness Considerations

### 6.1 Performance Optimization

**Current Implementation:**
- Indexes on `organizations.organization_name` and `admin_users.email`
- Connection pooling via Motor (async MongoDB driver)
- Pydantic validation reduces invalid data reaching database

**Production Enhancements:**
```python
# Add compound indexes for common queries
db.admin_users.create_index([("email", 1), ("organization_id", 1)])

# Implement caching layer (Redis)
@cache(ttl=300)
async def get_organization(name: str):
    return await org_repository.get_by_name(name)

# Query optimization with projections
db.organizations.find(
    {"organization_name": org_name},
    {"_id": 0, "created_at": 0}  # Exclude unused fields
)
```

### 6.2 Security Hardening

**Current Measures:**
- bcrypt password hashing (12 rounds, industry standard)
- JWT token expiration (configurable via `ACCESS_TOKEN_EXPIRE_MINUTES`)
- Input validation via Pydantic schemas

**Production Additions:**
- Rate limiting (prevent brute force attacks)
- Token revocation via Redis blocklist
- HTTPS enforcement
- CORS configuration for specific origins
- Audit logging for sensitive operations
- IP allowlisting for admin operations

### 6.3 Scalability Preparation

**Horizontal Scaling:**
```yaml
# Kubernetes deployment example
apiVersion: apps/v1
kind: Deployment
spec:
  replicas: 3  # Multiple API instances
  strategy:
    type: RollingUpdate
---
apiVersion: v1
kind: Service
spec:
  type: LoadBalancer  # Distribute traffic
```

**Database Scaling:**
- MongoDB sharding when collection size exceeds 100GB
- Read replicas for analytics workloads
- Separate read/write connection pools

**Monitoring:**
- APM integration (DataDog, New Relic)
- Health check endpoints (`/health`, `/readiness`)
- Distributed tracing for debugging multi-tenant issues

## 7. Implementation Summary

### Architecture Evaluation

Croupier's **collection-per-tenant architecture** provides an optimal balance for organizations scaling from hundreds to thousands of tenants. The implementation demonstrates production-ready patterns with clear growth paths.

**Technical Strengths:**
- **Layered Design:** Clean separation enables independent testing and maintenance
- **Type Safety:** Pydantic validation throughout reduces runtime errors
- **Security-First:** Industry-standard bcrypt + JWT patterns
- **Scalability:** Stateless design enables horizontal scaling

**Known Limitations:**
- Collection count scales to ≈10,000 tenants before architectural changes needed
- No token revocation mechanism (requires Redis integration)
- Cross-tenant analytics requires aggregation across collections

### Growth Path

```mermaid
graph LR
    A[Current: Collection/Tenant] --> B[Add Caching Layer]
    B --> C[Implement Sharding]
    C --> D[Hybrid Tiered Strategy]
    D --> E[Service Decomposition]
```

**Phase 1 (0-1k tenants):** Current architecture sufficient  
**Phase 2 (1k-10k tenants):** Add Redis caching, implement sharding  
**Phase 3 (10k+ tenants):** Migrate to hybrid strategy (shared/collection/database tiers)  
**Phase 4 (100k+ tenants):** Microservices decomposition, event-driven architecture

---

### Closing Notes

This architecture is **production-ready for initial deployment** and provides clear scaling paths. The modular design allows incremental improvements without requiring complete rewrites. With the documented enhancements, this system can grow from a startup MVP to an enterprise-scale platform serving hundreds of thousands of organizations.

**For deployment instructions, API reference, and quick start guide, see the main [README.md](README.md)**